<a href="https://colab.research.google.com/github/KevinTheRainmaker/ML_DL_Basics/blob/master/HonGong_ML_DL/16_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 심층 신경망

### **키워드:** 심층 신경망, ReLU 함수, 옵티마이저

케라스 인공 신경망에 은닉층을 추가하여 심층 신경망을 만들고 분류 과제를 수행해보자.

In [1]:
# packages
from tensorflow import keras

from sklearn.model_selection import train_test_split

## 데이터셋

In [15]:
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

train_scaled = X_train / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

train_scaled, val_scaled, y_train, y_val = train_test_split(train_scaled, y_train, test_size=0.2, random_state=42)

4431872/4422102 [==============================] - 0s 0us/step


## 은닉층

은닉층(hidden layer)란 입력층과 출력층 사이 존재하는 모든 층을 지칭하는 용어이다. 은닉층에 적용되는 활성화 함수는 출력층과 달리 다소 자유로운 편으로, 시그모이드 혹은 렐루(ReLU) 함수가 주로 이용된다.

은닉층에서 활성화 함수를 사용함으로써 선형 계산을 비선형 계산으로 틀어 계산의 복잡도와 정확도를 높인다.

In [2]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)) # 은닉층
dense2 = keras.layers.Dense(10, activation='softmax') # 출력층

`dense1`은 은닉층으로 100개의 뉴런을 가진 밀집층이다. 활성화 함수로 sigmoid를 지정했으며 `input_shape` 매개변수에서 입력의 크기를 (784,)로 지정했다.

은닉층의 뉴런 개수를 정하는 기준은 출쳑층의 뉴런보다 많아야 한다는 점 외에는 특별히 존재하지 않으며, 경험적으로 정해지곤 한다.

`dense1`는 출력층으로, 10개의 뉴런을 가지고 클래스를 분류한다. 다중 확률 값 중 최대치를 출력하기 위해 활성화 함수는 softmax로 설정하였다.

## 심층 신경망 만들기

앞서 만든 `dense1`과 `dense2` 객체를 Sequential 클래스에 추가하여 심층 신경망(Deep Neural Network, DNN)을 만들어보자.

In [5]:
model = keras.Sequential([dense1, dense2])

심층 신경망을 만들 때는 각 레이어를 순서대로 배치해야한다.

In [6]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


맨 첫 줄에 모델의 이름(sequential)이 나와있다. 

층마다 층 이름, 클래스, 출력 크기, 모델 파라미터 개수가 차례로 출력된다. 층을 만들 때 name 매개변수로 이름을 지정할 수 있는데, 따로 지정하지 않을 경우 자동으로 dense라는 이름이 붙는다.

출력 크기 (None, 100)의 앞쪽 None은 샘플의 개수를 나타내는데, None으로 나타나는 이유는 샘플의 개수가 정의되어 있지 않기 때문이다. 케라스 모델의 `fit()` 메서드가 기본적으로 32개의 미니배치를 사용하는 미니배치 경사하강법을 채택하는데, 이때의 미니배치의 사이즈는 `fit()` 메서드의 `batch_size` 매개변수로 바꿀 수 있다. 따라서 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정한다. 이렇게 신경망 층에 입력되거나 출력되는 배열의 첫 번째 차원을 배치 차원(batch dimension)이라고 부른다.

은닉층 또한 Dense층으로 설정했으므로 784 * 100 조합에 대한 가중치가 있고, 각 은닉층 뉴런에는 상응하는 절편이 있으며, 마지막 출력층에 대해서도 Dense층을 이루기 때문에 (784 \* 100 + 100) + (100 \* 10 + 10) = 79,510개의 훈련 파라미터(Trainable parmas)가 존재하게 된다.

아래의 Non-trainable params는 경사 하강법으로 훈련되지 않는 파라미터를 가진 층의 파라미터 개수이다.

In [12]:
# 다음과 같은 방식으로 레이어의 정의와 신경망 구축을 동시에 진행할 수도 있다.
model = keras.Sequential([
                          keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
                          keras.layers.Dense(10, activation='softmax', name='output')
], name='Fashion_MNIST_Model')

모델의 이름은 한글이어도 상관없으나, 층의 이름은 반드시 영문이어야한다.

In [13]:
model.summary()

Model: "Fashion_MNIST_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


층이 많아질 경우 위와 같은 방식으로 하기 불편해질 수 있는데, 이럴 경우 `add()` 메서드가 사용되곤 한다.

In [14]:
model = keras.Sequential(name='Fashion_MNIST_Model')
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax', name='output'))

model.summary()

Model: "Fashion_MNIST_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


이제 앞서 만든 모델을 훈련해보자.

In [17]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, y_train, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 6s 3ms/step - loss: 0.5605 - accuracy: 0.8082
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4068 - accuracy: 0.8531
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3726 - accuracy: 0.8648
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3492 - accuracy: 0.8730
Epoch 5/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3322 - accuracy: 0.8792


추가된 층이 성능을 향상시켰다. 층의 몇 개가 추가되는 사용법이 동일한 것은 케라스 API의 장점 중 하나이다.